# Tabulation of Fractional Cover data within shapefile polygons

**What does this notebook do?**

This notebook is a pilot collaboration between Geoscience Australia and Australian Bureau of Statistics. The purpose of the notebook is to use a shapefile polygon boundaries to load fractional cover dataset, complete zonal statistics and tabulate the results.

**Requirements**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles`

`module load dea/20181213`


**Background**

Data from the Landsat 5,7 and 8 satellite missions are accessible through Digital Earth Australia (DEA). The code snippets in this notebook will let you retrieve and plot the Fractional Cover (FC25) data stored in DEA.


**How to use this notebook**

A basic understanding of any programming language is desirable but one doesn't have to be an expert Python programmer to manipulate the code to get and display the data.This doc applies to the following Landsat satellites, Fractional Cover bands and the WOfS dataset:

- Landsat 5
- Landsat 7
- Landsat 8
- PV - Photosythetic vegetation
- NPV - Non-Photosythetic vegetation
- BS - Bare Soil
- UE - Unmixing Error
- Water Observations from Space (WOFs)
- WOfS Feature Layer (WOFL)

**Bugs still to fix**

- Memory errors for extra large polygons - AH & ET


**Errors or bugs**

If you find an error or bug in this notebook, please contact erin.telfer@ga.gov.au.


## 1. Import Libraries

In [1]:
%matplotlib inline

from datetime import time, datetime
import os.path

from matplotlib import pyplot as plt
import pandas as pd
import numpy
import xarray as xr
import rasterio
import rasterio.features
import fiona
from datetime import datetime
import dask
from dask.delayed import delayed
from dask.distributed import LocalCluster, Client
import tempfile

import datacube
from datacube import Datacube
from datacube.virtual import construct, construct_from_yaml
from datacube.ui.task_app import year_splitter

print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:37:40


## 2. Define directories and years of interest

In [ ]:
#Set folder  and SA2 shapefile locations
shapefile_path = '../input/SA_2016_edited_3577.shp'
output_path = '/g/data/r78/ext547/abs/output/'

In [ ]:
#Specify time years of interest
start_year, end_year = 2000, 2001
time_range = (str(start_year), str(end_year))
print(time_range)

## 3. Set up a local dask cluster
Some calculations take more memory than is available on a system.  By breaking the data up into chunks, we can chain a sequence of operations together, and work on the data a small piece at a time.

This lets several processes work at the same time, and manage total memory usage for the calculations.

In more advanced setups, we can distribute the work across multiple computers, using all of their memory and CPU power.

In [2]:
#Set up dask cluster
n_workers = 6
threads_per_worker=1
mem_per_worker = 8e9  # 8e9 is 8GB (8,000,000,000 bytes)

chunk_size = {'time': 1, 'x': 2000, 'y': 2000}

In [4]:
cluster = LocalCluster(local_dir=tempfile.gettempdir(), 
                       n_workers=n_workers, 
                       threads_per_worker=threads_per_worker,
                       memory_limit=mem_per_worker)
client = Client(cluster)
dask.config.set(get=client.get)
client

Client Scheduler: tcp://127.0.0.1:41471 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 6 Memory: 48.00 GB


We also get a dashboard to see how the system is running, by clicking the link above after the cell has been run.

## 4. Connect to the Datacube 

In [5]:
dc = Datacube()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:37:48


## 5. Construct the virtual product

In [6]:
#Remove Landsat 7 scenes with the Scan Line Correction (SLC) missing data
LS7_BROKEN_DATE = datetime(2003, 5, 31)
is_pre_slc_failure = lambda dataset: dataset.center_time < LS7_BROKEN_DATE
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:37:48


In [7]:
#Create function to ensure wofls in correct format
def wofls_fuser(dest, src):
    where_nodata = (src & 1) == 0
    numpy.copyto(dest, src, where=where_nodata)
    return dest
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:37:48


In [9]:
#Create virtual product so that datacube data can be loaded effectively within memory
fc_and_water_yaml = """
        juxtapose:
          - collate:
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls5_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls5_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls7_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                        # dataset_predicate: __main__.is_pre_slc_failure
                      - transform: make_mask
                        input:
                            product: ls7_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls8_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls8_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
          - transform: make_mask
            input:
                product: wofs_albers
                group_by: solar_day
                fuse_func: __main__.wofls_fuser
            flags:
                wet: true
            mask_measurement_name: water
"""
fc_and_water = construct_from_yaml(fc_and_water_yaml)

## 6. Set up functions

In [10]:
def geometry_mask(geoms, geobox, all_touched=False, invert=False, chunks=None):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    data = rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
    if chunks is not None:
        data = dask.array.from_array(data, chunks=tuple(chunks[d] for d in geobox.dims))
        
    coords = [xr.DataArray(data=coord.values, name=dim, dims=[dim], attrs={'units': coord.units}) 
              for dim, coord in geobox.coords.items()]
    return xr.DataArray(data, coords=coords)
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:37:51


In [11]:
def get_shapes(shape_file):
    with fiona.open(shape_file) as shapes:
        crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
        for shape in shapes:
            geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
            yield geom, shape['properties']
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:37:51


In [43]:
def fc_and_water_summary(data, mask_int):
    # Where there is water, set the FC bands to 0%
    fc = data[['PV', 'NPV', 'BS']].where(data.water==0, other=0)
    fc['water'] = data.water.where(numpy.isfinite(fc['BS'])) * numpy.float32(100)
    fc = fc.apply(lambda data_array: data_array.clip(0, 100).where(numpy.isfinite(data_array)))
    
    # Flatten to a monthly mean
    fc = fc.groupby(fc.time.astype('datetime64[M]'), squeeze=False).mean(dim='time', skipna=True)
    # Calculate percentage of cover based on area of mask
    percentage = fc.sum(dim=['x','y']) * (100 / mask_int)
    
    for da in percentage.data_vars.values():
        da.attrs['units'] = '%'

    fc_unclass = percentage.to_array('variable').sum(dim='variable')
    fc_unclass = 100 - float(fc_unclass.values)
    print(f"   PV = {int(percentage.PV.values)}%, NPV = {int(percentage.NPV.values)}%, BS = {int(percentage.BS.values)}%, water = {int(percentage.water.values)}%, unclassified = {int(fc_unclass)}%")
    
    fc['unclassified'] = ('time', numpy.repeat(fc_unclass,fc.time.size)) 
    percentage['unclassified'] = ('time', numpy.repeat(fc_unclass,fc.time.size)) 
        
    pixel_area_in_metres2 = 25 * 25
    m2_to_km2 = (1 / 1_000_000)
    percent_to_fraction = (1 / 100)
    
    area = (fc * (pixel_area_in_metres2 * m2_to_km2 * percent_to_fraction)).sum(dim=['x','y'])
    area = area.rename({'BS': 'BS_area', 
                        'PV': 'PV_area', 
                        'NPV': 'NPV_area', 
                        'water': 'water_area', 
                        'unclassified':'unclassified_area'})
    
    for da in area.data_vars.values():
        da.attrs['units'] = 'km^2'
    fc = percentage.merge(area)
    return fc

In [35]:
def keepna(a, dim=None, thresh=None):
    if type(a) is xr.Dataset:
        return a.apply(keepna, keep_attrs=True, dim=dim, thresh=thresh)
    
    keep_dim = [] if dim is None else [dim]
    dims = [d for d in a.dims if d not in keep_dim]
    if thresh is None:
        keep = numpy.isfinite(a).sum(dim=dims) > 0
    else:
        keep = numpy.isfinite(a).sum(dim=dims) >= thresh
    return a.where(keep, other=numpy.nan)

print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:50:10


In [36]:
def plot_stacked(ds, catchment_id,plot_title='title', show=True):
    if not show:
        plt.ioff()
        
    fig,ax = plt.subplots(figsize=(10,5))
    ax.stackplot(ds.dropna(dim='time').time.data, 
                 ds.dropna(dim='time').PV,
                 ds.dropna(dim='time').NPV, 
                 ds.dropna(dim='time').BS, 
                 ds.dropna(dim='time').water, 
                 ds.dropna(dim='time').unclassified, 
                 colors = ['darkolivegreen','olive','tan','darkblue','red'], 
                 labels=['PV','NPV','BS','Water','Unclassified',])
    plt.legend(loc='upper center', ncol = 5)
    plt.title(f'FC Components: Catchment ID {catchment_id}', size=12)
    plt.ylabel('Percentage (%)', size=12) #Set Y label
    plt.xlabel('Date', size=12) #Set X label
    
    plt.savefig(f'{output_path}/{catchment_id}_{plot_title}.png');
    plt.close(fig)
    
    # Turn interactive back on
    if not show:
        plt.show()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:50:10


In [37]:
def month_splitter(start_year, end_year_inclusive):
    yield from (str(p) for p in pd.period_range(start=pd.Period(start_year).start_time, 
                               end=pd.Period(end_year_inclusive).end_time, 
                               freq='M'))

## 7. Set up the query
For each year and polygon query the product, apply the geometry mask and compute the fractional cover stats

Using `client.compute()` lets us use the monthly results in calculating the annual results at the same time.

In [38]:
shape_file = os.path.expanduser(f'{shapefile_path}')
shapes = list(get_shapes(shape_file))

# Use this list instead of 'shapes' to just the big outback South Australian area
shapes = [(g,p) for g, p in shapes if str(p['SA2_MAIN16']) == '403041081']

print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-04 16:50:21


In [39]:
# start_year, end_year = 2000, 2001
# time_range = (str(start_year), str(end_year))
# print(time_range)
# print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

If we have enough resources, we can start the query and calculation of the next year's data while the previous is still being calculated. `by_slice=False` will be faster, but use more memory.

For larger areas `by_slice` will need to be `True`, so that the compute cluster does not become overwhelmed.  

If you get the error:
> `distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting`

then you will need to set `by_slice=True`

In [40]:
by_slice=True

In [41]:
def process_area(geometry, catchment_id, time_range):
    monthly_values = []
    annual_values = []
    mask = None
          
    for sub_time_range in month_splitter(time_range[0], time_range[-1]):
        print(sub_time_range)
        try:
            data = fc_and_water.load(dc, dask_chunks=chunk_size, 
                                     time=sub_time_range, 
                                     geopolygon=geometry)
        except ValueError:
            print(f'    No data for {sub_time_range} , skipping...')
            continue

        if mask is None:
            mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
            mask_int = mask * 1
            mask_int = mask_int.sum() * 100
        data = data.where(mask)
#         data = data.groupby(data.time.astype('datetime64[M]')).mean(dim='time', skipna=True)
#         data = keepna(data, dim='time', thresh=0.9*int(mask.sum()))

        monthly_data = fc_and_water_summary(data, mask_int)
#         monthly_data = monthly_data.where(monthly_data != 0)
        monthly_data = client.compute(monthly_data, sync=by_slice)
        monthly_values.append(monthly_data)
        
    if not by_slice:
        print("  all years queried, hard load data")
        monthly_values = client.gather(monthly_values)

    monthly_values = xr.concat(monthly_values, dim='time')
#     monthly_values = monthly_values.where(monthly_values[['PV', 'BS', 'NPV', 'water']].to_array(dim='var').sum(dim='var') > 90).dropna(dim='time')
    monthly_values = monthly_values.where(monthly_values['unclassified'] < 10).dropna(dim='time')
    annual_values = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
    print(f"  Calculation complete for annual values")

    
    plot_stacked(monthly_values, catchment_id, plot_title='monthly_plot_wofs',show=False)
    plot_stacked(annual_values, catchment_id, plot_title='annual_plot_wofs', show=False)
    
    print("  All data loaded, save to csv")
    monthly_values.to_dataframe().to_csv(f"{output_path}/{catchment_id}_monthly.csv")
    annual_values.to_dataframe().to_csv(f"{output_path}/{catchment_id}_annual.csv")
    print(f"  Catchment {catchment_id} done")

## 8. Process query

In [42]:
for geometry, properties in shapes:
    catchment_id = str(properties['SA2_MAIN16'])
    print(f"Catchment ID: {catchment_id}, size: {properties['AREASQKM16']}km^2, time: {time_range}")
    
    try:
          process_area(geometry, catchment_id, time_range)
    except Exception as e:
          print(f"Could not process {catchment_id}: {e}")
          client.restart()

Catchment ID: 403041081, size: 6.5089km^2, time: ('2000', '2001')
2000-01
 PV = 33%, NPV = 29%, BS = 9%, water = 25%, unclassified = 1%
2000-02
    No data for 2000-02 , skipping...
2000-03
 PV = 1%, NPV = 0%, BS = 0%, water = 0%, unclassified = 97%
2000-04
 PV = 43%, NPV = 24%, BS = 8%, water = 22%, unclassified = 0%
2000-05
 PV = 69%, NPV = 21%, BS = 6%, water = 0%, unclassified = 0%
2000-06
 PV = 6%, NPV = 1%, BS = 8%, water = 0%, unclassified = 83%
2000-07
 PV = 0%, NPV = 0%, BS = 0%, water = 0%, unclassified = 100%
2000-08
 PV = 3%, NPV = 0%, BS = 5%, water = 0%, unclassified = 90%
2000-09
 PV = 44%, NPV = 21%, BS = 8%, water = 25%, unclassified = 1%
2000-10
 PV = 2%, NPV = 0%, BS = 3%, water = 0%, unclassified = 92%
2000-11
 PV = 40%, NPV = 27%, BS = 8%, water = 23%, unclassified = 1%
2000-12
 PV = 35%, NPV = 29%, BS = 10%, water = 23%, unclassified = 1%
2001-01
 PV = 33%, NPV = 29%, BS = 12%, water = 23%, unclassified = 1%
2001-02
 PV = 34%, NPV = 32%, BS = 9%, water = 22%, uncl

In [ ]:
# def fc_summary(data, mask_int):
#     fc = data[['BS', 'PV', 'NPV']].sum(dim=('x', 'y'))
    
#     #calculate unclassified portion of SA2
#     fc_unclass = fc.to_array('variable').sum(dim='variable')
#     fc_unclass = mask_int - int(fc_unclass.values)
#     fc['UNCLASSIFIED'] = ('time', numpy.repeat(int(fc_unclass.values),fc.time.size)) 

#     #caclulate area of components 
#     area = fc * (25 * 25 / 1_000_000)
#     area = area.rename({'BS': 'BS_area', 'PV': 'PV_area', 'NPV': 'NPV_area','UNCLASSIFIED':'UNCLASSIFIED_area'})
    
#     for da in area.data_vars.values():
#         da.attrs['units'] = 'km2'

#     fc = fc / mask_int * 100
    
    
#     for da in fc.data_vars.values():
#         da.attrs['units'] = '%'
        
#     fc = fc.merge(area)
    
#     return fc

In [ ]:
# def fc_summary(data,mask_int):
#     fc = data[['BS', 'PV', 'NPV']].sum(dim=('x', 'y'))
    
#     area = fc * (25 * 25 / 1_000_000)
#     area = area.rename({'BS': 'BS_area', 'PV': 'PV_area', 'NPV': 'NPV_area'})
    
#     for da in area.data_vars.values():
#         da.attrs['units'] = 'km2'

#     fc = fc / mask_int * 100
#     fc_mean = fc.to_array('variable').sum(dim='variable')
    
#     fc_unclassified = 100 - int(fc_mean.values)
#     fc['UNCLASSIFIED'] = ('time', np.repeat(fc_unclassified,fc.time.size))

#     fc_unclassified_area = fc_unclassified  * (25 * 25 / 1_000_000) 
#     area['UNCLASSIFIED_area'] = ('time', np.repeat(fc_unclassified_area,fc.time.size))
    
#     for da in fc.data_vars.values():
#         da.attrs['units'] = '%'
        
#     fc = fc.merge(area)
    
#     return fc

In [ ]:
# def process_area(geometry, catchment_id, time_range):
#     monthly_values = []
#     annual_values = []
#     mask = None
          
#     for sub_time_range in month_splitter(time_range[0], time_range[-1]):
# #         print(f'  lazy loading {sub_time_range}...')
#         try:
#             data = fc_land_only.load(dc, dask_chunks=chunk_size, 
#                                      time=sub_time_range, 
#                                      geopolygon=geometry)
# #             print(f'    lazy loaded {sub_time_range}')
#         except ValueError:
#             print(f'    Error, skipping...')
#             continue

#         if mask is None:
#             mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
#             mask_int = mask * 1
#             mask_int = mask_int.sum() * 100
#         data = data.where(mask)
# #         print(client.compute([data], sync=by_slice))
#         data = data.resample(time='1MS').mean(dim='time', skipna=True)
#         data = keepna(data, dim='time', thresh=0.9*int(mask.sum()))

#         monthly_data = fc_summary(data,mask_int)
#         monthly_data = monthly_data.where(monthly_data != 0)
#         annual_data = monthly_data.resample(time='1YS').mean(dim='time', skipna=True)
#         print(f"    calculating for {dict(monthly_data.sizes)}")
#         monthly_data, annual_data = client.compute([monthly_data, annual_data], sync=by_slice)
# #         monthly_data, annual_data = client.compute([monthly_data, annual_data], sync=by_slice)

# #         print('data')
# #         print(data)
# #         print('mv')
# #         print(monthly_data)
# #         print('av')
# #         print(annual_values)   
#         monthly_values.append(monthly_data)
#         annual_values.append(annual_data)
        
#     if not by_slice:
#         print("  all years queried, hard load data")
#         monthly_values = client.gather(monthly_values)
#         annual_values = client.gather(annual_values)


#     print('b')
#     monthly_values = xr.concat(monthly_values, dim='time').dropna(dim='time')
#     annual_values02 = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
#     print('c')    
#     plot_stacked(monthly_values, catchment_id, show=False)
#     print('d')
#     annual_values = xr.concat(annual_values, dim='time').dropna(dim='time')
#     print('e')
    
#     print("  all data loaded, save to csv")
#     monthly_values.to_dataframe().to_csv(f"../output/{catchment_id}_monthly.csv")
#     annual_values.to_dataframe().to_csv(f"../output/{catchment_id}_annual.csv")
#     annual_values02.to_dataframe().to_csv(f"../output/{catchment_id}_annual02.csv")
#     return(monthly_values,annual_values,annual_values02)
#     print(f"  Catchment {catchment_id} done")